### 文本纠错
[官方文档](https://www.xfyun.cn/doc/nlp/textCorrection/API.html#%E8%B0%83%E7%94%A8%E7%A4%BA%E4%BE%8B)

In [6]:
#
# 文本纠错 WebAPI 接口调用示例
# 运行前：请先填写Appid、APIKey、APISecret、要纠错文本信息text
# 运行方法：直接运行 main 即可
# 结果： 控制台输出结果信息
#
# 接口文档（必看）：https://www.xfyun.cn/doc/nlp/textCorrection/API.html
#
from datetime import datetime
from wsgiref.handlers import format_date_time
from time import mktime
import hashlib
import base64
import hmac
from urllib.parse import urlencode
import os
import traceback
import json
import requests

class AssembleHeaderException(Exception):
    def __init__(self, msg):
        self.message = msg


class Url:
    def __init__(this, host, path, schema):
        this.host = host
        this.path = path
        this.schema = schema
        pass


# calculate sha256 and encode to base64
def sha256base64(data):
    sha256 = hashlib.sha256()
    sha256.update(data)
    digest = base64.b64encode(sha256.digest()).decode(encoding='utf-8')
    return digest


def parse_url(requset_url):
    stidx = requset_url.index("://")
    host = requset_url[stidx + 3:]
    schema = requset_url[:stidx + 3]
    edidx = host.index("/")
    if edidx <= 0:
        raise AssembleHeaderException("invalid request url:" + requset_url)
    path = host[edidx:]
    host = host[:edidx]
    u = Url(host, path, schema)
    return u


# build websocket auth request url
def assemble_ws_auth_url(requset_url, method="POST", api_key="", api_secret=""):
    u = parse_url(requset_url)
    host = u.host
    path = u.path
    now = datetime.now()
    date = format_date_time(mktime(now.timetuple()))
    #print(date)
    # date = "Thu, 12 Dec 2019 01:57:27 GMT"
    signature_origin = "host: {}\ndate: {}\n{} {} HTTP/1.1".format(host, date, method, path)
    #print(signature_origin)
    signature_sha = hmac.new(api_secret.encode('utf-8'), signature_origin.encode('utf-8'),
                             digestmod=hashlib.sha256).digest()
    signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')
    authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
        api_key, "hmac-sha256", "host date request-line", signature_sha)
    authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
    #print(authorization_origin)
    values = {
        "host": host,
        "date": date,
        "authorization": authorization
    }

    return requset_url + "?" + urlencode(values)

#控制台获取
APPId = "600ce683"
#控制台获取
APISecret = "d2832cd4d2b4ca57ce5f09855348e0c2"
#控制台获取
APIKey = "8bac623db639c11f4e5f7d262d61a5a3"
#需纠错文本
Text="明明明白白白的原因，但明明就想让白白把事情说的明明白白"



url = 'https://api.xf-yun.com/v1/private/s9a87e3ec'

body =  {
    "header": {
        "app_id": APPId,
        "status": 3
    },
    "parameter": {
        "s9a87e3ec": {
            "result": {
                "encoding": "utf8",
                "compress": "raw",
                "format": "json"
            }
        }
    },
    "payload": {
        "input": {
            "encoding": "utf8",
            "compress": "raw",
            "format": "plain",
            "status": 3,
            "text": base64.b64encode(Text.encode("utf-8")).decode('utf-8')
        }
    }
}

request_url = assemble_ws_auth_url(url, "POST", APIKey, APISecret)


headers = {'content-type': "application/json", 'host':'api.xf-yun.com', 'app_id':APPId}
#print(request_url)
response = requests.post(request_url, data = json.dumps(body), headers = headers)
#print(response)
print('response==>>\n' + response.content.decode())
tempResult=json.loads(response.content.decode())
#print(type(tempResult))
print('text==>>\n' + base64.b64decode(tempResult['payload']['result']['text']).decode())

response==>>
{"header":{"code":0,"message":"success","sid":"ase000e57ed@hu17732669b6f0210882"},"payload":{"result":{"compress":"raw","encoding":"utf8","format":"json","seq":"0","status":"3","text":"eyJjaGFyIjogW10sICJ3b3JkIjogW10sICJyZWR1bmQiOiBbWzMsICLnmb0iLCAiIiwgInJlZHVuZCJdXSwgIm1pc3MiOiBbXSwgIm9yZGVyIjogW10sICJkYXBlaSI6IFtdLCAicHVuYyI6IFtdLCAiaWRtIjogW10sICJvcmciOiBbXSwgImxlYWRlciI6IFtdLCAibnVtYmVyIjogW119"}}}
text==>>
{"char": [], "word": [], "redund": [[3, "白", "", "redund"]], "miss": [], "order": [], "dapei": [], "punc": [], "idm": [], "org": [], "leader": [], "number": []}


### 情感分析
[官方文档](https://www.xfyun.cn/doc/nlp/emotion-analysis/API.html#%E8%B0%83%E7%94%A8%E7%A4%BA%E4%BE%8B)

In [7]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import time
import urllib.request
import urllib.parse
import json
import hashlib
import base64
#接口地址
url ="http://ltpapi.xfyun.cn/v2/sa"
#开放平台应用ID
x_appid = "5ff87169"
#开放平台应用接口秘钥
api_key = "8cfbd3aa284fc8dbee117359155a8967"
#语言文本
TEXT="我不送彩礼，丈母娘不高兴了，我送彩礼，媳妇不独立了，我都不知道该保大还是保小？"


def main():
    body = urllib.parse.urlencode({'text': TEXT}).encode('utf-8')
    param = {"type": "dependent"}
    x_param = base64.b64encode(json.dumps(param).replace(' ', '').encode('utf-8'))
    x_time = str(int(time.time()))
    x_checksum = hashlib.md5(api_key.encode('utf-8') + str(x_time).encode('utf-8') + x_param).hexdigest()
    x_header = {'X-Appid': x_appid,
                'X-CurTime': x_time,
                'X-Param': x_param,
                'X-CheckSum': x_checksum}
    req = urllib.request.Request(url, body, x_header)
    result = urllib.request.urlopen(req)
    result = result.read()
    print(result.decode('utf-8'))
    return


if __name__ == '__main__':
    main()


{"code":"0","data":{"score":0.1486,"sentiment":-1},"desc":"success","sid":"ltp0079c9ee@dx65e5136833261aba00"}
